<a href="https://colab.research.google.com/github/YashNigam65/gitfolder/blob/master/assignment_3_audio_translator_with_5_languages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create a voice-to-voice translation application that allows users to select an input language and an output language from a predefined list of five languages, provide audio input (either by recording or uploading), translate the input audio's speech content into the selected output language, and then save the translated audio file to '/content/sample_data/{output_language}_result_audio'.

In [2]:
!pip install speechrecognition googletrans gTTS
!pip install pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1


In [3]:
import gradio as gr
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS
import os

# Define the list of languages for input/output selection
LANGUAGES = [
    'English',
    'Spanish',
    'French',
    'German',
    'Japanese',
    'Chinese',
    'Korean',
    'Hindi'
]

# Define a dictionary mapping language names to their ISO 639-1 codes
LANGUAGE_CODES = {
    'English': 'en',
    'Spanish': 'es',
    'French': 'fr',
    'German': 'de',
    'Japanese': 'ja',
    'Chinese': 'zh-cn',
    'Korean': 'ko',
    'Hindi': 'hi'
}

print("Libraries imported successfully.")
print("LANGUAGES and LANGUAGE_CODES defined.")

Libraries imported successfully.
LANGUAGES and LANGUAGE_CODES defined.


In [4]:
input_language_dropdown = gr.Dropdown(
    choices=LANGUAGES,
    label="Select Input Language",
    value=LANGUAGES[0] # Set a default value
)
print("Input language selection dropdown created.")

Input language selection dropdown created.


In [5]:
audio_input_component = gr.Audio(
    type="filepath",
    label="Input Audio",
    sources=["microphone", "upload"]
)
print("Gradio Audio input component created for recording and uploading.")

Gradio Audio input component created for recording and uploading.


In [6]:
output_language_dropdown = gr.Dropdown(
    choices=LANGUAGES,
    label="Select Output Language",
    value=LANGUAGES[0] # Set a default value
)
print("Output language selection dropdown created.")

Output language selection dropdown created.


Develop Audio Translation Function


In [9]:
from pydub import AudioSegment
from io import BytesIO

async def translate_audio(audio_filepath, input_lang, output_lang):
    recognizer = sr.Recognizer()

    try:
        # Load audio using pydub and convert to WAV in memory
        audio = AudioSegment.from_file(audio_filepath);
        wav_audio_data = BytesIO();
        audio.export(wav_audio_data, format="wav");
        wav_audio_data.seek(0) # Rewind the BytesIO object to the beginning

        with sr.AudioFile(wav_audio_data) as source:
            audio_data = recognizer.record(source)

        # Convert speech to text
        text = recognizer.recognize_google(audio_data, language=LANGUAGE_CODES[input_lang])
        print(f"Recognized text: {text}")

        # Translate text
        translator = Translator()
        translated_result = await translator.translate(text, src=LANGUAGE_CODES[input_lang], dest=LANGUAGE_CODES[output_lang])
        translated_text = translated_result.text
        print(f"Translated text: {translated_text}")

        # Convert translated text to speech
        tts = gTTS(text=translated_text, lang=LANGUAGE_CODES[output_lang], lang_check=False)

        # Define output path and save
        output_dir = "/content/sample_data"
        os.makedirs(output_dir, exist_ok=True)
        output_audio_filepath = os.path.join(output_dir, f"{output_lang}_translated_audio.mp3")
        tts.save(output_audio_filepath)
        print(f"Translated audio saved to: {output_audio_filepath}")

        return output_audio_filepath

    except sr.UnknownValueError:
        print("Speech Recognition could not understand audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Web Speech API service; {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

print("The 'translate_audio' function has been updated to use pydub for WAV conversion and handle async translation.")

The 'translate_audio' function has been updated to use pydub for WAV conversion and handle async translation.


In [11]:
iface = gr.Interface(
    fn=translate_audio,
    inputs=[
        audio_input_component,
        input_language_dropdown,
        output_language_dropdown
    ],
    outputs=gr.Audio(type="filepath", label="Translated Audio"),
    title="Voice-to-Voice Translator",
    description="Select input/output languages, provide audio, and get a translated audio file."
)

iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://359a5ab4658312a371.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Recognized text: hi guys how are you good afternoon I am just decide I'm recording this audio I will be used this audio in my assignment
Translated text: salut les gars, comment allez-vous, bon après-midi, je décide juste d'enregistrer cet audio. Je vais utiliser cet audio dans ma mission.
Translated audio saved to: /content/sample_data/French_translated_audio.mp3
Recognized text: hi guys how are you good afternoon I am just decide I'm recording this audio I will be used this audio in my assignment
Translated text: नमस्ते दोस्तों, आप कैसे हैं, शुभ दोपहर, मैंने अभी निर्णय लिया है कि मैं यह ऑडियो रिकॉर्ड कर रहा हूं, मैं इस ऑडियो का उपयोग अपने असाइनमेंट में करूंगा
Translated audio saved to: /content/sample_data/Hindi_translated_audio.mp3
Recognized text: hi guys how are you good afternoon I am just decide I'm recording this audio I will be used this audio in my assignment
Translated text: Hallo Leute, wie geht es euch? Guten Tag. Ich bin gerade dabei, dieses Audio aufzunehmen. Ich werde d